In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import zipfile
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

# zip file handler  
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip')
zip.extractall('/kaggle/temp/')

# list available files in the container
#print (zip.namelist())
'''import shutil
shutil.rmtree('/kaggle/working/train')'''

In [ ]:
filenames = os.listdir('/kaggle/temp/train') 
'''example: ['dog.890.jpg',
 'dog.1178.jpg',
 'dog.7845.jpg',
 'dog.4632.jpg',
 'cat.3660.jpg']'''
import re

category=[]
id = []
for name in filenames:
    id.append(name.split('.')[1])
    category.append(name.split('.')[0])
    '''if re.match(r"dog.*", name):
        category.append(1)
    else: 
        category.append(0)'''
df = pd.DataFrame.from_dict({'id':id,'filename':filenames,'category':category})
df.head()

In [ ]:
df.groupby('category').count()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, stratify=df.category)

In [ ]:
train.groupby('category').count()

In [ ]:
test.groupby('category').count()

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

'''train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_dataframe(train,directory='/kaggle/temp/train',x_col="filename", y_col="category",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_dataframe(test,directory='/kaggle/temp/train',x_col="filename", y_col="category",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')'''

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
'''# Building the CNN - 84.5% accuracy
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.2))
# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3
)
# Training the CNN on the Training set and evaluating it on the Test set
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[es])
history.history['val_loss']'''

In [ ]:
'''
#loss: 0.2263 - accuracy: 0.9010 - val_loss: 0.2897 - val_accuracy: 0.8783
# Building the CNN
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.2))
# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3
)
# Training the CNN on the Training set and evaluating it on the Test set
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[es])
history.history['val_loss']

'''

In [ ]:
'''predict_datagen = ImageDataGenerator(rescale = 1./255)
predict_set = predict_datagen.flow_from_dataframe(df,directory='/kaggle/temp/test1',x_col="filename", y_col=None,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = None,
                                            shuffle=False)
pred_out=cnn.predict_generator(predict_set,steps = df.shape[0])
training_set.class_indices
labels= [1 if output > 0.5 else 0 for output in pred_out]
#labels
df['label']=labels
df.drop(columns=['filename'],inplace=True)
df.to_csv('submission1.csv', index=False)'''

In [ ]:
#using Yolo for prediction
import os
import requests
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


download("https://pjreddie.com/media/files/yolov3.weights", dest_folder="/kaggle/temp/weights")
filenames = os.listdir('/kaggle/temp/weights') 
print(filenames)

In [ ]:
download("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", dest_folder="/kaggle/temp/weights")
filenames = os.listdir('/kaggle/temp/weights') 
print(filenames)

In [ ]:
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip')
zip.extractall('/kaggle/temp/')
filenames = os.listdir('/kaggle/temp/test1') 
''' 
'10435.jpg',
 '6790.jpg',
 '4644.jpg',
 '1835.jpg',
 '1102.jpg'
 '''
# list available files in the container
#print (zip.namelist())

In [ ]:
'''example: '''
import re
id = []
for name in filenames:
    id.append(name.split('.')[0])
    
df = pd.DataFrame.from_dict({'id':id,'filename':filenames})
df.head()

In [ ]:
# set of 80 class labels which YOLO is trained
class_labels = ["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
                "trafficlight","firehydrant","stopsign","parkingmeter","bench","bird","cat",
                "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
                "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sportsball",
                "kite","baseballbat","baseballglove","skateboard","surfboard","tennisracket",
                "bottle","wineglass","cup","fork","knife","spoon","bowl","banana","apple",
                "sandwich","orange","broccoli","carrot","hotdog","pizza","donut","cake","chair",
                "sofa","pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse",
                "remote","keyboard","cellphone","microwave","oven","toaster","sink","refrigerator",
                "book","clock","vase","scissors","teddybear","hairdrier","toothbrush"]

yolo_model = cv2.dnn.readNetFromDarknet('/kaggle/temp/weights/yolov3.cfg','/kaggle/temp/weights/yolov3.weights')
yolo_layers = yolo_model.getLayerNames()
yolo_output_layer = [yolo_layers[yolo_layer[0] - 1] for yolo_layer in yolo_model.getUnconnectedOutLayers()]

In [ ]:
def predict(image):
    img_to_detect = cv2.imread('/kaggle/temp/test1/'+image)
    img_height = img_to_detect.shape[0]
    img_width = img_to_detect.shape[1]
    # convert to blob to pass into model
    img_blob = cv2.dnn.blobFromImage(img_to_detect, 0.003922, (416, 416), swapRB=True, crop=False)
    # input preprocessed blob into model and pass through the model
    yolo_model.setInput(img_blob)
    # obtain the detection layers by forwarding through till the output layer
    obj_detection_layers = yolo_model.forward(yolo_output_layer)
    label = -1
    # loop over each of the layer outputs
    for object_detection_layer in obj_detection_layers:
        # loop over the detections
        for object_detection in object_detection_layer:

            # obj_detections[1 to 4] => will have the two center points, box width and box height
            # obj_detections[5] => will have scores for all objects within bounding box
            all_scores = object_detection[5:]
            predicted_class_id = np.argmax(all_scores)
            prediction_confidence = all_scores[predicted_class_id]

            # take only predictions with confidence more than 20%
            if prediction_confidence > 0.70:
                #get the predicted label
                predicted_class_label = class_labels[predicted_class_id]
                if(predicted_class_label=='cat'):
                    label= 0
                else: 
                    label= 1
    return label

In [ ]:
predict('10435.jpg')

In [ ]:
df['label']=df.filename.apply(lambda x: predict(x))

In [ ]:
df.head()

In [ ]:
os.remove('/kaggle/working/*.*') 

In [ ]:
df.drop(columns=['filename'],inplace=True)
df.to_csv('submission2.csv', index=False)